Récupération des données

In [2]:
import requests
from bs4 import BeautifulSoup as bs


req = requests.get('https://eplca.jrc.ec.europa.eu/EF-SDP/LCIAMethodList.xhtml;jsessionid=38C8EAA487296B1BB51D9AA1A12F3B43?stock=default')
list_indicators_soup = bs(req.content,'html.parser')


In [3]:
all_indicators = list_indicators_soup.find_all('tr', class_=lambda value: value and value.startswith('ui-widget-content ui-datatable'))


#for e in all_indicators:
    #print(e)



In [5]:
names, urls, second_urls = [], [], []

for i in all_indicators:
    # Récupère le nom et le l'URL de chaque indicateur
    data = i.find('a')
    names.append(data.text)
    link = "https://eplca.jrc.ec.europa.eu" + data.get("href")
    urls.append(link)

    # Utilise le lien pour refaire une soupe et trouver le 2ème lien qui contient les informations
    req = requests.get(link)
    soup = bs(req.content,'html.parser')
    second_i = soup.find('td', class_='datasetBlock ui-widget-content')
    second_data1 = second_i.find('a')
    second_data2 = second_data1.find_next('a')
    second_link = "https://eplca.jrc.ec.europa.eu" + second_data2.get("href")
    second_urls.append(second_link)



AttributeError: 'NoneType' object has no attribute 'find'

Vérification des liens et des noms que l'on a collecté

In [7]:
# Vérification des liens et des noms que l'on a collecté
print(names)
print(urls)
print(second_urls)

['EF-Acidification', 'EF-Climate change', 'EF-Climate change-Biogenic', 'EF-Climate change-Fossil', 'EF-Climate change-Land use and land use change', 'EF-Ecotoxicity, freshwater', 'EF-Eutrophication marine', 'EF-Eutrophication, freshwater', 'EF-Eutrophication, terrestrial', 'EF-Human toxicity, cancer']
['https://eplca.jrc.ec.europa.eu/EF-SDP/showLCIAMethod.xhtml;jsessionid=956A7DD50F4E0A178909259D7D55025A?uuid=b5c611c6-def3-11e6-bf01-fe55135034f3&stock=default', 'https://eplca.jrc.ec.europa.eu/EF-SDP/showLCIAMethod.xhtml;jsessionid=956A7DD50F4E0A178909259D7D55025A?uuid=b2ad6d9a-c78d-11e6-9d9d-cec0c932ce01&stock=default', 'https://eplca.jrc.ec.europa.eu/EF-SDP/showLCIAMethod.xhtml;jsessionid=956A7DD50F4E0A178909259D7D55025A?uuid=0db6bc32-3f72-48b9-bdb3-617849c2752f&stock=default', 'https://eplca.jrc.ec.europa.eu/EF-SDP/showLCIAMethod.xhtml;jsessionid=956A7DD50F4E0A178909259D7D55025A?uuid=2105d3ac-c7c7-4c80-b202-7328c14c66e8&stock=default', 'https://eplca.jrc.ec.europa.eu/EF-SDP/showLCIA

Récupération des XMLs + Récupération des données

In [48]:
req = requests.get(second_urls[0])
soup = bs(req.content,'lxml')

# Récupération de toutes les lignes du fichiers
rows = soup.find_all('factor')

# Initialisation des listes qui vont contenir les données
ids, references, locations, exchangedirections, meanvalues, status, comments = [], [], [], [], [], [], []

# Scrapping des données
for r in rows:
    ref = r.find('referencetoflowdataset')
    ids.append(ref['refobjectid'])
    references.append(ref.text)
    try: # Certains n'ont pas de location, j'ajoute un NaN
        locations.append(r.find('location').text)
    except:
        locations.append("NaN")

    exchangedirections.append(r.find('exchangedirection').text)
    meanvalues.append(r.find('meanvalue').text)
    status.append(r.find('dataderivationtypestatus').text)
    comments.append(r.find('generalcomment').text)




/home/ubuntuf/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [1]:
import pandas as pd

data = {'ids': ids,
        'references': references,
        'locations' : locations,
        'exchangedirections' : exchangedirections,
        'meanvalues' : meanvalues,
        'status' : status,
        'comments' : comments}


df_impacts = pd.DataFrame(data = data)
df_impacts

/tmp/ipykernel_9173/3157301892.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


NameError: name 'ids' is not defined